In [4]:
import streamlit as st
import os
from langchain_groq import ChatGroq
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFDirectoryLoader

from dotenv import load_dotenv

In [5]:
os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
groq_api_key=os.getenv('GROQ_API_KEY')


In [6]:
st.title('Your chats are recorded by Groq')

2024-10-15 17:34:13.578 
  command:

    streamlit run /home/beehyv/.local/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]


DeltaGenerator()

In [7]:
llm = ChatGroq(groq_api_key=groq_api_key,model_name='Llama3-8b-8192')
prompt = ChatPromptTemplate.from_template(
    """
    Answer the question based on provided context only
    <context>
    {context}
    <context>
    Questions = {input}
    """
)

In [9]:
import pandas as pd


# Read the CSV file into a Pandas DataFram

In [11]:
def vector_embeddings():
    if 'vectors' not in st.session_state:
        st.session_state.embeddings=OpenAIEmbeddings()
        st.session_state.loader=pd.read_csv('dollar.csv')
        st.session_state.docs = st.session.loader.load()
        st.session_state.text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
        st.session_state.final_documents=st.session_state.text_splitter.split_documents(st.session_state.docs[:20]) #splitting
        st.session_state.vectors=FAISS.from_documents(st.session_state.final_documents,st.session_state.embeddings) #vector OpenAI embeddings
        

In [12]:
prompt1=st.text_input("Enter Your Question From Doduments")


2024-10-15 17:51:04.755 Session state does not function when running a script without `streamlit run`


In [14]:
if st.button("Documents Embedding"):
    vector_embeddings()
    st.write("Vector Store DB Is Ready")

In [ ]:
import time
if prompt1:   
   
    document_chain=create_stuff_documents_chain(llm,prompt)
    retriever=st.session_state.vectors.as_retriever()
    retrieval_chain=create_retrieval_chain(retriever,document_chain)
    start=time.process_time()
    response=retrieval_chain.invoke({'input':prompt1})
    print("Response time :",time.process_time()-start)
    st.write(response['answer'])